## Part 1: 搭建一个分词工具

### Part 1.1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [27]:
import pandas as pd
import numpy as np
df = pd.read_excel('data/综合类中文词库.xlsx',header = 0)# TODO: 第一步： 从dic.txt中读取所有中文词。
#  hint: 思考一下用什么数据结构来存储这个词典会比较好？ 要考虑我们每次查询一个单词的效率。 
dic_words =  [i[0] for i in np.array(df.iloc[:,0:1]).tolist()]   # 保存词典库中读取的单词

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}
words = ['我们','是','经常','学习','人工智能']
#print (sum(word_prob.values()))


In [28]:
def full_segmentation(input_str,words):
    if input_str == '':
        return[[]]
    else:
        result = []
        for i in range(1,len(input_str)+1):
            if input_str[:i] in words:
                for remain_segment in full_segmentation(input_str[i:],words):
                    result.append([input_str[:i]] + remain_segment)
                return result

In [29]:
input_str = "我是经常学习人工智能"
result = full_segmentation(input_str,dic_words)
print(result)

[['我', '是', '经', '常', '学', '习', '人', '工', '智', '能']]


In [42]:
def check(input_str,dic_words):
    dp = [False for i in range(len(input_str)+1)]
    dp[0] = True
    for index in range(1,len(input_str)+1):    
        for i in range(0,index):
            if dp[i] is True and input_str[i:index] in dic_words:
                dp[index] = True
    return dp[len(input_str)]
segments = []# TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
             # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
            # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]
def word_segment_naive(input_str,strl=''):
    """
    对于输入字符串做分词，并返回所有可行的分词之后的结果。
    """
    
    if check(input_str,dic_words):
        if len(input_str) == 0:
                segments.append(strl[1:]) 
        for i in range(1, len(input_str)+1):
            if input_str[:i] in dic_words:
                word_segment_naive(input_str[i:],strl+" "+input_str[:i]) 
        
          
    return segments

In [43]:
# 测试
# print (word_segment_naive("北京的天气真好啊"))
# print (word_segment_naive("今天的课程内容很有意思"))
print (word_segment_naive("经常有意见分歧"))

['经 常 有 意 见 分 歧', '经 常 有 意 见 分歧', '经 常 有 意见 分 歧', '经 常 有 意见 分歧', '经 常 有意 见 分 歧', '经 常 有意 见 分歧', '经 常有 意 见 分 歧', '经 常有 意 见 分歧', '经 常有 意见 分 歧', '经 常有 意见 分歧', '经常 有 意 见 分 歧', '经常 有 意 见 分歧', '经常 有 意见 分 歧', '经常 有 意见 分歧', '经常 有意 见 分 歧', '经常 有意 见 分歧']


### Part 1.2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1.1的要求一致

In [31]:

word_prob.update({word: 0.00001 for word in dic_words if word not in word_prob.keys()})
word_score = {word:-math.log(word_prob[word]) for word  in word_prob.keys()}

In [36]:
import collections
import math


## TODO 请编写word_segment_viterbi函数来实现对输入字符串的分词
def get_graph(input_str):
    graph = collections.OrderedDict()
    for i in range(len(input_str)+1):
        if i not in graph:
            graph[i] = {}
        for j in range(i):
            if input_str[j:i] in word_score.keys():
                graph[i][j] = word_score[input_str[j:i]]
    return graph

    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    
    # TODO: 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph） 参考：课程内容
    #      有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率在 word_prob，如果不在word_prob里的单词但在
    #      词典里存在的，统一用概率值0.00001。
    #      注意：思考用什么方式来存储这种有向图比较合适？ 不一定有只有一种方式来存储这种结构。 
#     graph = 
    
    # TODO： 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    #              hint: 思考为什么不用相乘: p(w1)p(w2)...而是使用negative log sum:  -log(w1)-log(w2)-...
def get_best_path(graph):
    score = {0:0}
    path = {}
    for i in graph.keys():
        if graph[i]:
            for j in graph[i].keys():
                if i not in score.keys() or score[i] > graph[i][j] + score[j]:
                    score[i] = graph[i][j] + score[j]
                    path[i] = j
    return path
# TODO: 第三步： 根据最好的PATH, 返回最好的切分
def path_to_segment(path,input_str):
    segment = []
    i = max(path.keys())
    while i>0:
        segment = [input_str[path[i]:i]] + segment
        i = path[i]
    return segment


def word_segment_viterbi(input_str):
    graph = get_graph(input_str)
    print(graph)
    path = get_best_path(graph)
    print(path)
    best_segment = path_to_segment(path,input_str)
    return best_segment
   

In [46]:
print(word_segment_viterbi("今天的课程内容很有意思"))

OrderedDict([(0, {}), (1, {0: 4.605170185988091}), (2, {0: 2.659260036932778, 1: 5.298317366548036}), (3, {2: 2.5257286443082556}), (4, {3: 4.605170185988091}), (5, {3: 2.8134107167600364, 4: 5.298317366548036}), (6, {5: 11.512925464970229}), (7, {5: 2.8134107167600364, 6: 11.512925464970229}), (8, {7: 3.506557897319982}), (9, {7: 3.2188758248682006, 8: 2.995732273553991}), (10, {8: 11.512925464970229, 9: 4.605170185988091}), (11, {8: 5.298317366548036, 9: 2.8134107167600364, 10: 11.512925464970229})])
{1: 0, 2: 0, 3: 2, 4: 3, 5: 3, 6: 5, 7: 5, 8: 7, 9: 7, 10: 9, 11: 9}
['今天', '的', '课程', '内容', '很有', '意思']
